# **Obtención de registro de recorridos de taxis**

#### Para obtener los datos de recorridos de taxis amarillos y verdes en la Ciudad de New York, realizaremos Web Scraping a la [página](http://www.nyc.gov/html/tlc/html/about/trip_record_data.shtml) provista en el [diccionario de datos](Datasets/data_dictionary_trip_records_yellow.pdf).

### Importamos las librerías necesarias.

In [1]:
import requests
from bs4 import BeautifulSoup
from Funciones import *
import os
import re
import pandas as pd

### Efectuamos Web Scrapping para obtener los enlaces de registros de viajes en taxis.

In [2]:
# Guardamos la URL en una variable
url = 'https://www.nyc.gov/site/tlc/about/tlc-trip-record-data.page'

# Realizamos la solicitud HTTP
response = requests.get(url)

# Verificamos si la solicitud fue exitosa
if response.status_code == 200:
    # Analizamos el contenido HTML con BeautifulSoup
    soup = BeautifulSoup(response.text, 'html.parser')

    # Buscamos el elemento <div> con la clase "faq-answers" y el id "faq2023"
    target_div = soup.find('div', {'class': 'faq-answers', 'id': 'faq2023'})

    if target_div:
        # Buscamos los enlaces <a> dentro de target_div que tienen title="Yellow Taxi Trip Records" o "Green Taxi Trip Records"
        taxi_links = target_div.find_all('a', title=['Yellow Taxi Trip Records', 'Green Taxi Trip Records'])

        if taxi_links:
            print("Enlaces con el título 'Yellow Taxi Trip Records' o 'Green Taxi Trip Records':")
            links = []
            for link in taxi_links:
                links.append(link.get('href'))
                print(link.get('href'))  # Imprime el enlace URL
        else:
            print("No se encontraron enlaces con los títulos 'Yellow Taxi Trip Records' o 'Green Taxi Trip Records' dentro de 'faq-answers'.")

    else:
        print("No se encontró el elemento con la clase 'faq-answers' y el id 'faq2023'.")

else:
    print("La solicitud no fue exitosa. Código de estado:", response.status_code)

Enlaces con el título 'Yellow Taxi Trip Records' o 'Green Taxi Trip Records':
https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-01.parquet
https://d37ci6vzurychx.cloudfront.net/trip-data/green_tripdata_2023-01.parquet
https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-02.parquet
https://d37ci6vzurychx.cloudfront.net/trip-data/green_tripdata_2023-02.parquet
https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-03.parquet
https://d37ci6vzurychx.cloudfront.net/trip-data/green_tripdata_2023-03.parquet 
https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-04.parquet
https://d37ci6vzurychx.cloudfront.net/trip-data/green_tripdata_2023-04.parquet
https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-05.parquet 
https://d37ci6vzurychx.cloudfront.net/trip-data/green_tripdata_2023-05.parquet 
https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-06.parquet
https://d37ci6vzurychx.cloudfront.net/trip-d

### Habiendo visualizado la estructura de los enlaces, procedemos a crear una carpeta donde se almacenarán los archivos que descargaremos.

In [3]:
# Creamos una arpeta de destino para guardar los archivos descargados
carpeta_destino = "Datasets/2-yellow_and_green_taxis/"

# Verificamos si la carpeta de destino existe, si no existe la creamos
if not os.path.exists(carpeta_destino):
    os.makedirs(carpeta_destino)

# Obtenemos la lista de archivos en la carpeta de destino
archivos_locales = os.listdir(carpeta_destino)

# Generamos dinámicamente los enlaces para los meses del 01 al 10
meses = ['01', '02', '03', '04', '05', '06', '07', '08', '09', '10']

# Guardamos la URL en una variable
url = "https://www.nyc.gov/site/tlc/about/tlc-trip-record-data.page"

# Realizamos la solicitud HTTP a la página
response = requests.get(url)

# Verificamos si la solicitud fue exitosa (código de estado 200)
if response.status_code == 200:
    # Parsear el contenido HTML de la página
    soup = BeautifulSoup(response.text, 'html.parser')

    # Encontramos todos los enlaces que contienen 'yellow' o 'green' y tienen el formato esperado para 2023
    links_pagina = {a['href'] for a in soup.find_all('a', href=True) if re.match(r'^.*(yellow|green)_tripdata_2023-\d{2}\.parquet$', a['href'])}

    # Combinamos enlaces generados dinámicamente con los obtenidos de la página
    links_totales = links_pagina.copy()  # Creamos una copia para mantener los enlaces originales
    for mes in meses:
        enlace_yellow = f"https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-{mes}.parquet"
        enlace_green = f"https://d37ci6vzurychx.cloudfront.net/trip-data/green_tripdata_2023-{mes}.parquet"
        links_totales.add(enlace_yellow)
        links_totales.add(enlace_green)

    # Realizamos la descarga de los archivos
    for link in sorted(links_totales):
        nombre_archivo = link.split("/")[-1]  # Obtenemos el nombre del archivo desde la URL
        archivo_destino = os.path.join(carpeta_destino, nombre_archivo)

        if nombre_archivo not in archivos_locales:
            response = requests.get(link)

            if response.status_code == 200:
                with open(archivo_destino, 'wb') as archivo:
                    archivo.write(response.content)
                print(f"Archivo descargado: {nombre_archivo}")
            else:
                print(f"No se pudo descargar el archivo: {nombre_archivo}. Código de estado: {response.status_code}")
        else:
            print(f"El archivo {nombre_archivo} ya existe localmente.")
else:
    print(f"No se pudo acceder a la página. Código de estado: {response.status_code}")


El archivo green_tripdata_2023-01.parquet ya existe localmente.
El archivo green_tripdata_2023-02.parquet ya existe localmente.
El archivo green_tripdata_2023-03.parquet ya existe localmente.
El archivo green_tripdata_2023-04.parquet ya existe localmente.
El archivo green_tripdata_2023-05.parquet ya existe localmente.
El archivo green_tripdata_2023-06.parquet ya existe localmente.
El archivo green_tripdata_2023-07.parquet ya existe localmente.
El archivo green_tripdata_2023-08.parquet ya existe localmente.
El archivo green_tripdata_2023-09.parquet ya existe localmente.
Archivo descargado: green_tripdata_2023-10.parquet
El archivo yellow_tripdata_2023-01.parquet ya existe localmente.
El archivo yellow_tripdata_2023-02.parquet ya existe localmente.
El archivo yellow_tripdata_2023-03.parquet ya existe localmente.
El archivo yellow_tripdata_2023-04.parquet ya existe localmente.
El archivo yellow_tripdata_2023-05.parquet ya existe localmente.
El archivo yellow_tripdata_2023-06.parquet ya ex

### _**Unimos los dataframes mensuales de taxis verdes en un solo dataframe.**_

In [4]:
enero_g = pd.read_parquet("Datasets/2-yellow_and_green_taxis/green_tripdata_2023-01.parquet")
febrero_g = pd.read_parquet("Datasets/2-yellow_and_green_taxis/green_tripdata_2023-02.parquet")
marzo_g = pd.read_parquet("Datasets/2-yellow_and_green_taxis/green_tripdata_2023-03.parquet")
abril_g = pd.read_parquet("Datasets/2-yellow_and_green_taxis/green_tripdata_2023-04.parquet")
mayo_g = pd.read_parquet("Datasets/2-yellow_and_green_taxis/green_tripdata_2023-05.parquet")
junio_g = pd.read_parquet("Datasets/2-yellow_and_green_taxis/green_tripdata_2023-06.parquet")
julio_g = pd.read_parquet("Datasets/2-yellow_and_green_taxis/green_tripdata_2023-07.parquet")
agosto_g = pd.read_parquet("Datasets/2-yellow_and_green_taxis/green_tripdata_2023-08.parquet")
septiembre_g = pd.read_parquet("Datasets/2-yellow_and_green_taxis/green_tripdata_2023-09.parquet")
octubre_g = pd.read_parquet("Datasets/2-yellow_and_green_taxis/green_tripdata_2023-10.parquet")

In [5]:
# Listamos los DataFrames
dfs = [enero_g, febrero_g, marzo_g, abril_g, mayo_g, junio_g, julio_g, agosto_g, septiembre_g, octubre_g]

# Concateamos para crear uno solo
green_taxis = pd.concat(dfs, ignore_index=True)
green_taxis.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 658820 entries, 0 to 658819
Data columns (total 20 columns):
 #   Column                 Non-Null Count   Dtype         
---  ------                 --------------   -----         
 0   VendorID               658820 non-null  int64         
 1   lpep_pickup_datetime   658820 non-null  datetime64[us]
 2   lpep_dropoff_datetime  658820 non-null  datetime64[us]
 3   store_and_fwd_flag     612829 non-null  object        
 4   RatecodeID             612829 non-null  float64       
 5   PULocationID           658820 non-null  int64         
 6   DOLocationID           658820 non-null  int64         
 7   passenger_count        612829 non-null  float64       
 8   trip_distance          658820 non-null  float64       
 9   fare_amount            658820 non-null  float64       
 10  extra                  658820 non-null  float64       
 11  mta_tax                658820 non-null  float64       
 12  tip_amount             658820 non-null  floa

In [6]:
# Guardamos columnas a eliminar en una variable
columnas_eliminar = ['store_and_fwd_flag', 'fare_amount', 'extra', 'mta_tax', 'tolls_amount', 'improvement_surcharge',
                    'ehail_fee', 'congestion_surcharge', 'trip_type', 'VendorID', 'RatecodeID', 'payment_type']

# Eliminamos las columnas de la lista
green_taxis.drop(columns=columnas_eliminar, inplace=True)

# Renombramos las columnas
green_taxis.rename(columns={'lpep_pickup_datetime': 'pickup_datetime','lpep_dropoff_datetime': 'dropoff_datetime',
                            'total_amount': 'amount'}, inplace=True)
green_taxis.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 658820 entries, 0 to 658819
Data columns (total 8 columns):
 #   Column            Non-Null Count   Dtype         
---  ------            --------------   -----         
 0   pickup_datetime   658820 non-null  datetime64[us]
 1   dropoff_datetime  658820 non-null  datetime64[us]
 2   PULocationID      658820 non-null  int64         
 3   DOLocationID      658820 non-null  int64         
 4   passenger_count   612829 non-null  float64       
 5   trip_distance     658820 non-null  float64       
 6   tip_amount        658820 non-null  float64       
 7   amount            658820 non-null  float64       
dtypes: datetime64[us](2), float64(4), int64(2)
memory usage: 40.2 MB


In [7]:
green_taxis.head()

,pickup_datetime,dropoff_datetime,PULocationID,DOLocationID,passenger_count,trip_distance,tip_amount,amount
0,2023-01-01 00:26:10,2023-01-01 00:37:11,166,143,1.0,2.58,4.03,24.18
1,2023-01-01 00:51:03,2023-01-01 00:57:49,24,43,1.0,1.81,2.64,15.84
2,2023-01-01 00:35:12,2023-01-01 00:41:32,223,179,1.0,0.00,1.94,11.64
3,2023-01-01 00:13:14,2023-01-01 00:19:03,41,238,1.0,1.30,1.70,10.20
4,2023-01-01 00:33:04,2023-01-01 00:39:02,41,74,1.0,1.10,0.00,8.00


### En New York la propina (tip_amount) se considera una costumbre. Conforme a lo detallado en el diccionario provisto, el pago total incluye impuestos pero no propina asi que procedemos a sumar las columnas 'amount' y 'tip_amount'.

In [8]:
# Sumamos las columnas amount y tip_amount para crear la columna total_amount
green_taxis['total_amount'] = green_taxis['amount'] + green_taxis['tip_amount']

# Mostrar las primeras filas del DataFrame resultante
green_taxis.head()

,pickup_datetime,dropoff_datetime,PULocationID,DOLocationID,passenger_count,trip_distance,tip_amount,amount,total_amount
0,2023-01-01 00:26:10,2023-01-01 00:37:11,166,143,1.0,2.58,4.03,24.18,28.21
1,2023-01-01 00:51:03,2023-01-01 00:57:49,24,43,1.0,1.81,2.64,15.84,18.48
2,2023-01-01 00:35:12,2023-01-01 00:41:32,223,179,1.0,0.00,1.94,11.64,13.58
3,2023-01-01 00:13:14,2023-01-01 00:19:03,41,238,1.0,1.30,1.70,10.20,11.90
4,2023-01-01 00:33:04,2023-01-01 00:39:02,41,74,1.0,1.10,0.00,8.00,8.00


In [9]:
# Eliminamos las columnas amount y tip_amount al no ser necesarias
green_taxis.drop(columns=['amount', 'tip_amount'], inplace=True)
green_taxis.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 658820 entries, 0 to 658819
Data columns (total 7 columns):
 #   Column            Non-Null Count   Dtype         
---  ------            --------------   -----         
 0   pickup_datetime   658820 non-null  datetime64[us]
 1   dropoff_datetime  658820 non-null  datetime64[us]
 2   PULocationID      658820 non-null  int64         
 3   DOLocationID      658820 non-null  int64         
 4   passenger_count   612829 non-null  float64       
 5   trip_distance     658820 non-null  float64       
 6   total_amount      658820 non-null  float64       
dtypes: datetime64[us](2), float64(3), int64(2)
memory usage: 35.2 MB


In [10]:
# Revisamos el contenido de la columna passenger_count
green_taxis['passenger_count'].value_counts()

passenger_count
1.0    518701
2.0     53147
5.0     16342
3.0      8627
6.0      7698
0.0      5173
4.0      3063
7.0        41
8.0        22
9.0        15
Name: count, dtype: int64

In [11]:
# Verificamos nulos
green_taxis['passenger_count'].isnull().sum()

45991

In [12]:
# Verificamos la composicion del DataFrame cuando passenger_count es nulo para no tomar decisiones erroneas
verificar = green_taxis[green_taxis['passenger_count'].isnull()]
verificar.head()

,pickup_datetime,dropoff_datetime,PULocationID,DOLocationID,passenger_count,trip_distance,total_amount
63887,2023-01-01 00:28:00,2023-01-01 00:54:00,7,17,NaN,9.33,51.85
63888,2023-01-01 00:39:00,2023-01-01 00:52:00,37,37,NaN,2.67,21.77
63889,2023-01-01 00:58:00,2023-01-01 01:21:00,80,7,NaN,7.39,46.39
63890,2023-01-01 00:35:00,2023-01-01 00:55:00,169,42,NaN,4.04,18.85
63891,2023-01-01 00:45:00,2023-01-01 00:55:00,42,262,NaN,3.86,32.78


### Teniendo en cuenta que hay ocasiones en que no se informa de cantidad de pasajeros pero si hay un registro de pago, procedemos a completar los valores nulos de la columna 'passenger_count' con el valor 1.

In [13]:
# Completamos los valores faltantes de la columna passenger_count con el valor 1
green_taxis['passenger_count'].fillna(1, inplace=True)
green_taxis['passenger_count'].isnull().sum()

0

In [14]:
# Eliminamos las filas donde passenger_count es igual a 0.0
green_taxis = green_taxis[green_taxis['passenger_count'] != 0.0]
green_taxis['passenger_count'].value_counts()

passenger_count
1.0    564692
2.0     53147
5.0     16342
3.0      8627
6.0      7698
4.0      3063
7.0        41
8.0        22
9.0        15
Name: count, dtype: int64

In [15]:
# Revisamos el contenido de la columna trip_distance
green_taxis['trip_distance'].value_counts()

trip_distance
0.00         32549
1.40          5595
1.30          5305
1.20          4677
1.50          4569
             ...  
64.45            1
20.86            1
1082.45          1
120098.84        1
11235.11         1
Name: count, Length: 3697, dtype: int64

In [16]:
# Eliminamos las filas donde trip_distance es igual a 0.00
green_taxis = green_taxis[green_taxis['trip_distance'] != 0.00]
green_taxis['trip_distance'].value_counts()

trip_distance
1.40         5595
1.30         5305
1.20         4677
1.50         4569
1.10         4243
             ... 
101197.87       1
20.51           1
7353.65         1
10805.53        1
11235.11        1
Name: count, Length: 3696, dtype: int64

In [17]:
# Creamos una columna llamada service_type con el valor 'green'
green_taxis['service_type'] = 'green'
green_taxis.head(2)

,pickup_datetime,dropoff_datetime,PULocationID,DOLocationID,passenger_count,trip_distance,total_amount,service_type
0,2023-01-01 00:26:10,2023-01-01 00:37:11,166,143,1.0,2.58,28.21,green
1,2023-01-01 00:51:03,2023-01-01 00:57:49,24,43,1.0,1.81,18.48,green


### _**Unimos los dataframes mensuales de taxis amarillos en un solo dataframe.**_

In [18]:
enero_y = pd.read_parquet("Datasets/2-yellow_and_green_taxis/yellow_tripdata_2023-01.parquet")
febrero_y = pd.read_parquet("Datasets/2-yellow_and_green_taxis/yellow_tripdata_2023-02.parquet")
marzo_y = pd.read_parquet("Datasets/2-yellow_and_green_taxis/yellow_tripdata_2023-03.parquet")
abril_y = pd.read_parquet("Datasets/2-yellow_and_green_taxis/yellow_tripdata_2023-04.parquet")
mayo_y = pd.read_parquet("Datasets/2-yellow_and_green_taxis/yellow_tripdata_2023-05.parquet")
junio_y = pd.read_parquet("Datasets/2-yellow_and_green_taxis/yellow_tripdata_2023-06.parquet")
julio_y = pd.read_parquet("Datasets/2-yellow_and_green_taxis/yellow_tripdata_2023-07.parquet")
agosto_y = pd.read_parquet("Datasets/2-yellow_and_green_taxis/yellow_tripdata_2023-08.parquet")
septiembre_y = pd.read_parquet("Datasets/2-yellow_and_green_taxis/yellow_tripdata_2023-09.parquet")
octubre_y = pd.read_parquet("Datasets/2-yellow_and_green_taxis/yellow_tripdata_2023-10.parquet")

In [19]:
# Listamos los DataFrames
dfs = [enero_y, febrero_y, marzo_y, abril_y, mayo_y, junio_y, julio_y, agosto_y, septiembre_y, octubre_y]

# Concatenamos para crear uno solo
yellow_taxis = pd.concat(dfs, ignore_index=True)
yellow_taxis.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 31593944 entries, 0 to 31593943
Data columns (total 20 columns):
 #   Column                 Dtype         
---  ------                 -----         
 0   VendorID               int64         
 1   tpep_pickup_datetime   datetime64[us]
 2   tpep_dropoff_datetime  datetime64[us]
 3   passenger_count        float64       
 4   trip_distance          float64       
 5   RatecodeID             float64       
 6   store_and_fwd_flag     object        
 7   PULocationID           int64         
 8   DOLocationID           int64         
 9   payment_type           int64         
 10  fare_amount            float64       
 11  extra                  float64       
 12  mta_tax                float64       
 13  tip_amount             float64       
 14  tolls_amount           float64       
 15  improvement_surcharge  float64       
 16  total_amount           float64       
 17  congestion_surcharge   float64       
 18  airport_fee         

In [20]:
# Guardamos columnas a eliminar en una variable
columnas_eliminar = ['store_and_fwd_flag', 'fare_amount', 'extra', 'mta_tax', 'tolls_amount', 'improvement_surcharge',
                    'congestion_surcharge', 'airport_fee', 'Airport_fee', 'VendorID', 'RatecodeID', 'payment_type']

# Eliminamos las columnas de la lista
yellow_taxis.drop(columns=columnas_eliminar, inplace=True)

# Renombramos las columnas
yellow_taxis.rename(columns={'tpep_pickup_datetime': 'pickup_datetime','tpep_dropoff_datetime': 'dropoff_datetime',
                             'total_amount': 'amount'}, inplace=True)
yellow_taxis.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 31593944 entries, 0 to 31593943
Data columns (total 8 columns):
 #   Column            Dtype         
---  ------            -----         
 0   pickup_datetime   datetime64[us]
 1   dropoff_datetime  datetime64[us]
 2   passenger_count   float64       
 3   trip_distance     float64       
 4   PULocationID      int64         
 5   DOLocationID      int64         
 6   tip_amount        float64       
 7   amount            float64       
dtypes: datetime64[us](2), float64(4), int64(2)
memory usage: 1.9 GB


In [21]:
yellow_taxis.head()

,pickup_datetime,dropoff_datetime,passenger_count,trip_distance,PULocationID,DOLocationID,tip_amount,amount
0,2023-01-01 00:32:10,2023-01-01 00:40:36,1.0,0.97,161,141,0.00,14.30
1,2023-01-01 00:55:08,2023-01-01 01:01:27,1.0,1.10,43,237,4.00,16.90
2,2023-01-01 00:25:04,2023-01-01 00:37:49,1.0,2.51,48,238,15.00,34.90
3,2023-01-01 00:03:48,2023-01-01 00:13:25,0.0,1.90,138,7,0.00,20.85
4,2023-01-01 00:10:29,2023-01-01 00:21:19,1.0,1.43,107,79,3.28,19.68


### En New York la propina (tip_amount) se considera una costumbre. Conforme a lo detallado en el diccionario provisto, el pago total incluye impuestos pero no propina asi que procedemos a sumar las columnas 'amount' y 'tip_amount'.

In [22]:
# Sumamos las columnas amount y tip_amount para crear la columna total_amount
yellow_taxis['total_amount'] = yellow_taxis['amount'] + yellow_taxis['tip_amount']

# Mostrar las primeras filas del DataFrame resultante
yellow_taxis.head()

,pickup_datetime,dropoff_datetime,passenger_count,trip_distance,PULocationID,DOLocationID,tip_amount,amount,total_amount
0,2023-01-01 00:32:10,2023-01-01 00:40:36,1.0,0.97,161,141,0.00,14.30,14.30
1,2023-01-01 00:55:08,2023-01-01 01:01:27,1.0,1.10,43,237,4.00,16.90,20.90
2,2023-01-01 00:25:04,2023-01-01 00:37:49,1.0,2.51,48,238,15.00,34.90,49.90
3,2023-01-01 00:03:48,2023-01-01 00:13:25,0.0,1.90,138,7,0.00,20.85,20.85
4,2023-01-01 00:10:29,2023-01-01 00:21:19,1.0,1.43,107,79,3.28,19.68,22.96


In [23]:
# Eliminamos las columnas amount y tip_amount al no ser necesarias
yellow_taxis.drop(columns=['amount', 'tip_amount'], inplace=True)
yellow_taxis.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 31593944 entries, 0 to 31593943
Data columns (total 7 columns):
 #   Column            Dtype         
---  ------            -----         
 0   pickup_datetime   datetime64[us]
 1   dropoff_datetime  datetime64[us]
 2   passenger_count   float64       
 3   trip_distance     float64       
 4   PULocationID      int64         
 5   DOLocationID      int64         
 6   total_amount      float64       
dtypes: datetime64[us](2), float64(3), int64(2)
memory usage: 1.6 GB


In [24]:
# Revisamos los valores de la columna passenger_count
yellow_taxis['passenger_count'].value_counts()

passenger_count
1.0    23038865
2.0     4611264
3.0     1143944
4.0      631774
0.0      499112
5.0      404224
6.0      267804
8.0         166
7.0          67
9.0          46
Name: count, dtype: int64

In [25]:
# Verificamos nulos
yellow_taxis['passenger_count'].isnull().sum()

996678

In [26]:
# Verificamos la composición del DataFrame cuando passenger_count es nulo para no tomar decisiones erroneas
verificar2 = yellow_taxis[yellow_taxis['passenger_count'].isnull()]
verificar2.head()

,pickup_datetime,dropoff_datetime,passenger_count,trip_distance,PULocationID,DOLocationID,total_amount
2995023,2023-01-01 00:02:40,2023-01-01 00:30:36,NaN,0.00,142,79,24.13
2995024,2023-01-01 00:57:48,2023-01-01 01:12:34,NaN,0.00,237,263,24.10
2995025,2023-01-01 00:55:37,2023-01-01 01:00:41,NaN,0.86,24,41,18.58
2995026,2023-01-01 00:43:47,2023-01-01 00:56:12,NaN,5.36,88,229,37.07
2995027,2023-01-01 00:34:00,2023-01-01 00:51:00,NaN,6.03,232,263,50.82


### Teniendo en cuenta que hay ocasiones en que no se informa de cantidad de pasajeros pero si hay un registro de pago, procedemos a completar los valores nulos de la columna 'passenger_count' con el valor 1.

In [27]:
# Completamos los valores faltantes de la columna passenger_count con el valor 1
yellow_taxis['passenger_count'].fillna(1, inplace=True)
yellow_taxis['passenger_count'].isnull().sum()

0

In [28]:
# Eliminamos las filas donde passenger_count es igual a 0.0
yellow_taxis = yellow_taxis[yellow_taxis['passenger_count'] != 0.0]
yellow_taxis['passenger_count'].value_counts()

passenger_count
1.0    24035543
2.0     4611264
3.0     1143944
4.0      631774
5.0      404224
6.0      267804
8.0         166
7.0          67
9.0          46
Name: count, dtype: int64

In [29]:
# Revisamos los valores de la columna trip_distance
yellow_taxis['trip_distance'].value_counts()

trip_distance
0.00       584574
1.00       405061
0.90       403986
1.10       395261
0.80       391019
            ...  
52.06           1
61.97           1
71.25           1
69.80           1
4547.48         1
Name: count, Length: 8157, dtype: int64

In [30]:
# Eliminamos las filas donde trip_distance es igual a 0.00
yellow_taxis = yellow_taxis[yellow_taxis['trip_distance'] != 0.00]
yellow_taxis['trip_distance'].value_counts()

trip_distance
1.00       405061
0.90       403986
1.10       395261
0.80       391019
1.20       383900
            ...  
9679.47         1
48.04           1
65.09           1
52.21           1
4547.48         1
Name: count, Length: 8156, dtype: int64

In [31]:
# Creamos una columna llamada service_type con el valor 'yellow'
yellow_taxis['service_type'] = 'yellow'
yellow_taxis.head(2)

,pickup_datetime,dropoff_datetime,passenger_count,trip_distance,PULocationID,DOLocationID,total_amount,service_type
0,2023-01-01 00:32:10,2023-01-01 00:40:36,1.0,0.97,161,141,14.3,yellow
1,2023-01-01 00:55:08,2023-01-01 01:01:27,1.0,1.10,43,237,20.9,yellow


### Teniendo los dataframes de taxis amarillos y verdes y habiendo creado una columna que los distinga, procedemos a unirlos en un solo dataframe.

In [32]:
# Concatenamos los DataFrames de yellow y green taxis
taxis = pd.concat([yellow_taxis, green_taxis], ignore_index=True)
taxis.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 31131356 entries, 0 to 31131355
Data columns (total 8 columns):
 #   Column            Dtype         
---  ------            -----         
 0   pickup_datetime   datetime64[us]
 1   dropoff_datetime  datetime64[us]
 2   passenger_count   float64       
 3   trip_distance     float64       
 4   PULocationID      int64         
 5   DOLocationID      int64         
 6   total_amount      float64       
 7   service_type      object        
dtypes: datetime64[us](2), float64(3), int64(2), object(1)
memory usage: 1.9+ GB


In [33]:
# Aplicamos la función para crear nuevas columnas
taxis['pickup_borough'] = taxis['PULocationID'].apply(get_borough)
taxis['dropoff_borough'] = taxis['DOLocationID'].apply(get_borough)

In [34]:
taxis.pickup_borough.value_counts()

pickup_borough
Manhattan        27321484
Queens            3167940
Unknown            300380
Brooklyn           286741
Bronx               52047
Staten Island        1817
EWR                   947
Name: count, dtype: int64

In [35]:
taxis.dropoff_borough.value_counts()

dropoff_borough
Manhattan        27334748
Queens            1794793
Brooklyn          1284248
Unknown            413020
Bronx              202820
EWR                 92679
Staten Island        9048
Name: count, dtype: int64

In [36]:
# Eliminamos todas las filas donde pickup_borough o dropoff_borough sea igual a 'Unknown'
taxis = taxis[(taxis['pickup_borough'] != 'Unknown') & (taxis['dropoff_borough'] != 'Unknown')]
taxis.info()

<class 'pandas.core.frame.DataFrame'>
Index: 30648503 entries, 0 to 31131355
Data columns (total 10 columns):
 #   Column            Dtype         
---  ------            -----         
 0   pickup_datetime   datetime64[us]
 1   dropoff_datetime  datetime64[us]
 2   passenger_count   float64       
 3   trip_distance     float64       
 4   PULocationID      int64         
 5   DOLocationID      int64         
 6   total_amount      float64       
 7   service_type      object        
 8   pickup_borough    object        
 9   dropoff_borough   object        
dtypes: datetime64[us](2), float64(3), int64(2), object(3)
memory usage: 2.5+ GB


In [ ]:
# Cambiamos el tipo de dato de 'passenger_count' a int
taxis['passenger_count'] = taxis['passenger_count'].astype(int)
taxis.info()

### Guardamos el dataframe de taxis en un archivo parquet.

In [36]:
taxis.to_parquet("Datasets/3-Normalizados/taxis_2023.parquet")

In [ ]:
import pandas as pd
import boto3
from io import BytesIO
from botocore.exceptions import NoCredentialsError

def read_parquet_from_s3(bucket_name, key):
    s3 = boto3.client('s3')
    try:
        response = s3.get_object(Bucket=bucket_name, Key=key)
        parquet_data = response['Body'].read()
        df = pd.read_parquet(BytesIO(parquet_data))
        return df
    except NoCredentialsError:
        print('Credentials not available')

def etl_taxis(df, service_type):
    # Guardamos columnas a eliminar en una variable
    columnas_eliminar = ['store_and_fwd_flag', 'fare_amount', 'extra', 'mta_tax', 'tolls_amount', 'improvement_surcharge',
                        'ehail_fee', 'congestion_surcharge', 'trip_type', 'VendorID', 'RatecodeID', 'payment_type']

    # Eliminamos las columnas de la lista
    df.drop(columns=columnas_eliminar, inplace=True)

    # Renombramos las columnas
    df.rename(columns={'lpep_pickup_datetime': 'pickup_datetime', 'lpep_dropoff_datetime': 'dropoff_datetime',
                       'tpep_pickup_datetime': 'pickup_datetime', 'tpep_dropoff_datetime': 'dropoff_datetime',
                       'total_amount': 'amount'}, inplace=True)

    # Sumamos las columnas amount y tip_amount para crear la columna total_amount
    df['total_amount'] = df['amount'] + df['tip_amount']

    # Eliminamos las columnas amount y tip_amount al no ser necesarias
    df.drop(columns=['amount', 'tip_amount'], inplace=True)

    # Completamos los valores faltantes de la columna passenger_count con el valor 1
    df['passenger_count'].fillna(1, inplace=True)

    # Eliminamos las filas donde passenger_count es igual a 0.0
    df = df[df['passenger_count'] != 0.0]

    # Eliminamos las filas donde trip_distance es igual a 0.00
    df = df[df['trip_distance'] != 0.00]

    # Creamos una columna llamada service_type con el valor proporcionado
    df['service_type'] = service_type

    return df

def get_borough(location_id):
    # ... (código de la función get_borough)
    return 'Unknown'

def lambda_handler(event, context):
    bucket_name = "web-scrap-bucket"
    service_types = ['yellow', 'green']
    dfs = []

    for service_type in service_types:
        service_dfs = []
        for month in range(1, 13):
            file_key = f"{service_type}_tripdata_2023-{str(month).zfill(2)}.parquet"
            df = read_parquet_from_s3(bucket_name, file_key)
            df_transformed = etl_taxis(df, service_type)
            service_dfs.append(df_transformed)

        taxis_data = pd.concat(service_dfs, ignore_index=True)

        dfs.append(taxis_data)

    # Concatenar los DataFrames de amarillo y verde
    taxis = pd.concat(dfs, ignore_index=True)

    # Aplicar la función para crear nuevas columnas
    taxis['pickup_borough'] = taxis['PULocationID'].apply(get_borough)
    taxis['dropoff_borough'] = taxis['DOLocationID'].apply(get_borough)

    # Eliminar filas donde pickup_borough o dropoff_borough sea igual a 'Unknown'
    taxis = taxis[(taxis['pickup_borough'] != 'Unknown') & (taxis['dropoff_borough'] != 'Unknown')]

    # Cambiar el tipo de dato de 'passenger_count' a int
    taxis['passenger_count'] = taxis['passenger_count'].astype(int)

    # Guardar el DataFrame en un archivo parquet dentro del bucket de S3
    taxis.to_parquet("s3://web-scrap-bucket/taxis_2023.parquet")
    print("ETL process completed.")
